In [1]:
import requests
import pandas as pd
import csv

## Read Station Data

In [2]:
# Read stations data to be converted to FIPS.
stations = []  # List of format (station_id, latitude, longitude)
with open('data/ghcnd-stations.txt') as f:
    for line in f:
        # Starts getting messy after state, but we just need station id and longitude/latitude, so disregard the rest.
        columns = tuple(line.split()[:3])
        
        # Only care about the US stations.
        if columns[0][0:2] == 'US':
            stations.append(columns)

stations[-1]

('USW00096409', '68.6483', '-149.3986')

## Convert Station Latitude/Longitude to FIPS
Uses the following API from FCC: https://geo.fcc.gov/api/census/#!/block/get_block_find

In [3]:
# Reference: https://gis.stackexchange.com/questions/294641/python-code-for-transforming-lat-long-into-fips-codes

i = 0
for station in stations:
    # Display progress.
    print('\r', 'Progress: {}%'.format(str(round(i / (len(stations)-1) * 100, 3))), end='')
    
    station_id = station[0]
    lat = station[1]
    lon = station[2]
    
    # Contruct request URL
    url = 'https://geo.fcc.gov/api/census/block/find?latitude={}&longitude={}&censusYear=2020&format=json'.format(lat, lon)
    
    # Get response from API
    response = requests.get(url)

    # Parse json in response
    data = response.json()

    # Extract location data.
    fips = data['County']['FIPS']
    county = data['County']['name']
    state_code = data['State']['code']
    state = data['State']['name']

    # Save to CSV.
    f = open('output/location_data.csv', 'a')
    f.write('{},{},{},{},{},{},{}\n'.format(station_id, fips, county, state_code, state, lat, lon))
    f.close()
    
    # Simply for progress tracking.
    i += 1

 Progress: 100.0%%

In [4]:
# Below is just to track time of completion.
# Script took roughly 3 hours to complete: datetime.datetime(2024, 3, 21, 2, 44, 39, 974410)
import datetime
now = datetime.datetime.now()
now

datetime.datetime(2024, 3, 21, 2, 44, 39, 974410)